In [2]:
import pandas as pd
from datetime import datetime
import pickle
import sqlite3

In [3]:
# Load the file layout
layout = pd.read_excel("../Data/mortgage_data/file_layout.xlsx", sheet_name=None)

## Column name extraction from Freddie Mac documentation

In [4]:
# sheet_names = layout.keys()
# Extract column names and data types for both origination and performance datasets
orig_layout = layout['Origination Data File']
perf_layout = layout['Monthly Performance Data File']

# Extract column names and data types
orig_column_names = orig_layout['ATTRIBUTE NAME'].tolist()
orig_data_types = orig_layout['DATA TYPE & FORMAT'].tolist()
perf_column_names = perf_layout['ATTRIBUTE NAME'].tolist()
perf_data_types = perf_layout['DATA TYPE & FORMAT'].tolist()

cols_keep_perf = perf_layout['KEEP'].tolist()
cols_keep_orig = orig_layout['KEEP'].tolist()


## Load data per year

In [5]:
def drop_cols_and_NAN(data):
    #first drop all columns that only have NaN values
    data = data.dropna(axis=1, how='all')
    #drop cols_to_drop
    return data

In [6]:
#function that drops columns where cols_keep is 0
def drop_cols(data, cols_keep, col_names):
    cols_to_drop = []
    for i in range(len(cols_keep)):
        if cols_keep[i] == 0:
            cols_to_drop.append(col_names[i])
    data = data.drop(cols_to_drop, axis=1)
    return data

In [7]:
def load_yearly_data(year, base_dir="../Data/mortgage_data"):
    """
    Load and format the origination and performance datasets for a given year, considering the folder structure.
    
    Parameters:
    - year: The year for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - orig_data: Formatted origination dataset for the given year.
    - perf_data: Formatted performance dataset for the given year.
    """


    # Construct file paths considering the "sample_YYYY" folder structure
    orig_file_path = f"{base_dir}/sample_{year}/sample_orig_{year}.txt"
    perf_file_path = f"{base_dir}/sample_{year}/sample_svcg_{year}.txt"
    
    # Load origination data
    orig_data = pd.read_csv(orig_file_path, sep="|", header=None, low_memory=False)
    #select only the first 22 columns
    orig_data = orig_data.iloc[:, 0:22]
    #rename columns according to orig_column_names first 22
    orig_data.columns = orig_column_names[0:22]
    
    # Load performance data
    perf_data = pd.read_csv(perf_file_path, sep="|", header=None, names=perf_column_names, low_memory=False)
    
    try:
        orig_data = drop_cols(orig_data, cols_keep_orig[0:22], orig_column_names)
        perf_data = drop_cols(perf_data, cols_keep_perf, perf_column_names)
        # display('cols dropped')
    except:
        # display('no cols dropped')
        pass
    orig_data = drop_cols_and_NAN(orig_data)
    perf_data = drop_cols_and_NAN(perf_data)
    return orig_data, perf_data


## Load all data at once into dictionary

In [8]:
def load_all_datasets(start_year=1999, end_year=2022, base_dir="../Data/mortgage_data/"):
    """
    Load all origination and performance datasets for a given range of years.
    
    Parameters:
    - start_year: The starting year (inclusive) for which to load the data.
    - end_year: The ending year (inclusive) for which to load the data.
    - base_dir: The base directory where the datasets are stored.
    
    Returns:
    - datasets: Dictionary containing formatted origination and performance datasets for the given range of years.
    """
    
    datasets = {}
    
    for year in range(start_year, end_year + 1):
        display(year)
        orig_data, perf_data = load_yearly_data(year, base_dir=base_dir)
        datasets[f"orig_{year}"] = orig_data
        # display(orig_data.shape)
        datasets[f"perf_{year}"] = perf_data
        # display(perf_data.shape)
    
    return datasets

# For demonstration purposes, we'll load only the 2022 sample data
# To load all years' data, you would simply call load_all_datasets() without the year range
datasets_tot = load_all_datasets(start_year=1999, end_year=2022)
datasets_tot.keys()  # Display the keys of the dictionary

1999

2000

2001

2002

2003

...

2019

2020

2021

2022

dict_keys(['orig_1999', 'perf_1999', 'orig_2000', 'perf_2000', 'orig_2001', 'perf_2001', 'orig_2002', 'perf_2002', 'orig_2003', 'perf_2003', 'orig_2004', 'perf_2004', 'orig_2005', 'perf_2005', 'orig_2006', 'perf_2006', 'orig_2007', 'perf_2007', 'orig_2008', 'perf_2008', 'orig_2009', 'perf_2009', 'orig_2010', 'perf_2010', 'orig_2011', 'perf_2011', 'orig_2012', 'perf_2012', 'orig_2013', 'perf_2013', 'orig_2014', 'perf_2014', 'orig_2015', 'perf_2015', 'orig_2016', 'perf_2016', 'orig_2017', 'perf_2017', 'orig_2018', 'perf_2018', 'orig_2019', 'perf_2019', 'orig_2020', 'perf_2020', 'orig_2021', 'perf_2021', 'orig_2022', 'perf_2022'])

In [9]:
def merge_orig_with_perf(orig_data, perf_data):
    merged_data = pd.merge(perf_data, orig_data, on="LSN", how="left")
    #move Loan Sequence Number to the front
    merged_data = merged_data[["LSN"] + [col for col in merged_data.columns if col != "LSN"]]
    #move MRP to the front
    merged_data = merged_data[["MRP"] + [col for col in merged_data.columns if col != "MRP"]]
    return merged_data

In [10]:
def merge_all_datasets(datasets):
    """
    Merge all origination and performance datasets within the provided dictionary according to their year.
    
    Parameters:
    - datasets: Dictionary containing formatted origination and performance datasets.
    
    Returns:
    - merged_datasets: Dictionary containing merged datasets for each year.
    """
    merged_datasets = {}
    # Extract the range of years from the dataset keys
    years = sorted(set(int(key.split("_")[-1]) for key in datasets.keys()))
    for year in years:
        orig_key = f"orig_{year}"
        perf_key = f"perf_{year}"
        if orig_key in datasets and perf_key in datasets:
            merged_data = merge_orig_with_perf(datasets[orig_key], datasets[perf_key])
            merged_data['Date'] = merged_data['MRP'].astype(str)
            merged_datasets[f"fm_{year}"] = merged_data
            print("merged", year)
    return merged_datasets

# Merge all datasets in the provided dictionary (in this case, datasets_demo)
merged_datasets = merge_all_datasets(datasets_tot)
merged_datasets.keys()  # Display the keys of the merged datasets dictionary


merged 1999
merged 2000
merged 2001
merged 2002
merged 2003
merged 2004
merged 2005
merged 2006
merged 2007
merged 2008
merged 2009
merged 2010
merged 2011
merged 2012
merged 2013
merged 2014
merged 2015
merged 2016
merged 2017
merged 2018
merged 2019
merged 2020
merged 2021
merged 2022


dict_keys(['fm_1999', 'fm_2000', 'fm_2001', 'fm_2002', 'fm_2003', 'fm_2004', 'fm_2005', 'fm_2006', 'fm_2007', 'fm_2008', 'fm_2009', 'fm_2010', 'fm_2011', 'fm_2012', 'fm_2013', 'fm_2014', 'fm_2015', 'fm_2016', 'fm_2017', 'fm_2018', 'fm_2019', 'fm_2020', 'fm_2021', 'fm_2022'])

In [12]:
fm_1999 = merged_datasets['fm_1999']
display(fm_1999.shape)
display(fm_1999)

(2502944, 18)

,MRP,LSN,CLDS,CIR,ELTV,DDD,CS,FPD,FIRST_F,MD,CLTV,DTI,LTV,OIR,P_TYPE,POSTAL,OLT,Date
0,200209,F99Q10000029,0,6.375,NaN,NaN,618,200210,N,202902,85,24,85,6.375,SF,44200,317,200209
1,200210,F99Q10000029,0,6.375,NaN,NaN,618,200210,N,202902,85,24,85,6.375,SF,44200,317,200210
2,200211,F99Q10000029,0,6.375,NaN,NaN,618,200210,N,202902,85,24,85,6.375,SF,44200,317,200211
3,200212,F99Q10000029,0,6.375,NaN,NaN,618,200210,N,202902,85,24,85,6.375,SF,44200,317,200212
4,200301,F99Q10000029,0,6.375,NaN,NaN,618,200210,N,202902,85,24,85,6.375,SF,44200,317,200301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502939,201408,F99Q40245148,0,7.050,NaN,NaN,674,200612,N,201412,80,50,59,7.050,SF,72100,97,201408
2502940,201409,F99Q40245148,0,7.050,NaN,NaN,674,200612,N,201412,80,50,59,7.050,SF,72100,97,201409
2502941,201410,F99Q40245148,0,7.050,NaN,NaN,674,200612,N,201412,80,50,59,7.050,SF,72100,97,201410
2502942,201411,F99Q40245148,0,7.050,NaN,NaN,674,200612,N,201412,80,50,59,7.050,SF,72100,97,201411


In [ ]:
del datasets_tot
del cols_keep_orig
del cols_keep_perf
del layout
del orig_column_names
del orig_data_types
del perf_column_names
del perf_data_types
del perf_layout
del orig_layout
del fm_1999


Drop Columns and add 3ZiP

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
2502939   NaN
2502940   NaN
2502941   NaN
2502942   NaN
2502943   NaN
Name: ELTV, Length: 2502944, dtype: float64

In [ ]:
for key in merged_datasets.keys():
    merged_datasets[key]['3ZIP'] = merged_datasets[key]['POSTAL'].astype(str).str[:3].astype('int16')
    #Transform DDD to 0 if NaN and 1 if Y
    merged_datasets[key]['DDD'] = merged_datasets[key]['DDD'].fillna(0)
    merged_datasets[key]['DDD'] = merged_datasets[key]['DDD'].replace('Y', 1)
    #Transform FIRST_F to 0 if N and 1 if Y
    merged_datasets[key]['FIRST_F'] = merged_datasets[key]['FIRST_F'].replace('N', 0)
    merged_datasets[key]['FIRST_F'] = merged_datasets[key]['FIRST_F'].replace('Y', 1)
    #Convert int64 to int32 or int16 or bool
    merged_datasets[key]['DDD'] = merged_datasets[key]['DDD'].astype('bool')
    merged_datasets[key]['FIRST_F'] = merged_datasets[key]['FIRST_F'].astype('bool')
    merged_datasets[key]['ELTV'] = merged_datasets[key]['ELTV'].astype('Int16')
    merged_datasets[key]['CS'] = merged_datasets[key]['CS'].astype('Int16')
    merged_datasets[key]['CLTV'] = merged_datasets[key]['CLTV'].astype('Int16')
    merged_datasets[key]['OLT'] = merged_datasets[key]['OLT'].astype('Int16')
    merged_datasets[key]['DTI'] = merged_datasets[key]['DTI'].astype('Int16')
    merged_datasets[key]['FPD'] = merged_datasets[key]['FPD'].astype('Int32')
    merged_datasets[key]['MD'] = merged_datasets[key]['MD'].astype('int32')
    #Drop POSTAL and MRP
    merged_datasets[key].drop(['POSTAL'], axis=1, inplace=True)
    merged_datasets[key].drop(['MRP'], axis=1, inplace=True)
    #Move Date and 3ZIP to the front
    merged_datasets[key] = merged_datasets[key][["Date", "3ZIP"] + [col for col in merged_datasets[key].columns if col not in ["Date", "3ZIP"]]]
    print(f"{key} added and dropped")
merged_datasets['fm_2022'].head()

fm_1999 added and dropped
fm_2000 added and dropped
fm_2001 added and dropped
fm_2002 added and dropped
fm_2003 added and dropped
fm_2004 added and dropped
fm_2005 added and dropped
fm_2006 added and dropped
fm_2007 added and dropped
fm_2008 added and dropped
fm_2009 added and dropped
fm_2010 added and dropped
fm_2011 added and dropped
fm_2012 added and dropped
fm_2013 added and dropped
fm_2014 added and dropped
fm_2015 added and dropped
fm_2016 added and dropped
fm_2017 added and dropped
fm_2018 added and dropped
fm_2019 added and dropped
fm_2020 added and dropped
fm_2021 added and dropped
fm_2022 added and dropped


,Date,3ZIP,LSN,CLDS,CIR,ELTV,DDD,CS,FPD,FIRST_F,MD,CLTV,DTI,LTV,OIR,P_TYPE,OLT
0,202202,125,F22Q10000012,0,2.625,57,False,768,202203,False,203702,57,28,57,2.625,SF,180
1,202203,125,F22Q10000012,0,2.625,48,False,768,202203,False,203702,57,28,57,2.625,SF,180
2,202204,125,F22Q10000012,0,2.625,52,False,768,202203,False,203702,57,28,57,2.625,SF,180
3,202205,125,F22Q10000012,0,2.625,40,False,768,202203,False,203702,57,28,57,2.625,SF,180
4,202206,125,F22Q10000012,0,2.625,39,False,768,202203,False,203702,57,28,57,2.625,SF,180


In [ ]:
display(merged_datasets['fm_2022'].dtypes)
display(merged_datasets['fm_2022']['DTI'].describe())

Date        object
3ZIP         int32
LSN         object
CLDS         int64
CIR        float64
ELTV         Int16
DDD           bool
CS           Int16
FPD          Int32
FIRST_F       bool
MD           int32
CLTV         Int16
DTI          Int16
LTV          int64
OIR        float64
P_TYPE      object
OLT          Int16
dtype: object

count     419440.0
mean     36.752868
std      15.328947
min            1.0
25%           30.0
50%           38.0
75%           44.0
max          999.0
Name: DTI, dtype: Float64

In [ ]:
with open("../Data/mortgage_data/fm_datasets.pickle", "wb") as f:
    pickle.dump(merged_datasets, f)

# From here you can work with fm_datasets

In [ ]:
import pandas as pd
from datetime import datetime
import pickle
import sqlite3

In [ ]:

with open("../Data/mortgage_data/fm_datasets.pickle", "rb") as f:
    fm_datasets = pickle.load(f)

## Function to merge macro data to performance

In [ ]:
def merge_macro_data(perf_data, macro_data, perf_date_col, macro_date_col):
    """
    Merge macroeconomic data with the performance dataset based on the date.
    
    Parameters:
    - perf_data: The performance dataset.
    - macro_data: The macroeconomic dataset.
    - perf_date_col: The date column name in the performance dataset.
    - macro_date_col: The date column name in the macroeconomic dataset.
    
    Returns:
    - Merged dataset.
    """
    
    # Merge datasets based on the date
    merged_data = pd.merge(perf_data, macro_data, left_on=perf_date_col, right_on=macro_date_col, how='left')
    return merged_data

# Sample usage of the functions can be provided if datasets are available.
# For now, these functions are generic and can be adapted to actual data.

In [ ]:
def extract_year(value):
    value = str(value)
    if len(value) == 4:
        return value
    else:
        try:
            # Handle dates like "2022-01-15"
            return datetime.strptime(value, "%Y-%m-%d").year
        except:
            try:
                # Handle dates like "202201"
                return datetime.strptime(value, "%Y%m").year
            except:
                return value

def extract_month(value):
    value = str(value)
    try:
        # Handle month names like "Dec", "Jan", etc.
        return datetime.strptime(value, "%b").month
    except:
        try:
            # Handle dates like "2022-01-15"
            return datetime.strptime(value, "%Y-%m-%d").month
        except:
            try:
                # Handle dates like "202201" or "20221"
                return datetime.strptime(value, "%Y%m").month
            except:
                return value  #or some default value
#function that takes month column and year column and returns a "YYYYmm" string

# Access Database

In [12]:
import sqlite3
# Create a database connection
db_path = "../Database/thesis_database.db"
conn = sqlite3.connect(db_path)

In [ ]:
for key, dataset in fm_datasets.items():
    print("Writing", key, "to database...")
    dataset.to_sql(key, conn, if_exists = "replace", index = False)

Writing fm_1999 to database...
Writing fm_2000 to database...
Writing fm_2001 to database...
Writing fm_2002 to database...
Writing fm_2003 to database...
Writing fm_2004 to database...
Writing fm_2005 to database...
Writing fm_2006 to database...
Writing fm_2007 to database...
Writing fm_2008 to database...
Writing fm_2009 to database...
Writing fm_2010 to database...
Writing fm_2011 to database...
Writing fm_2012 to database...
Writing fm_2013 to database...
Writing fm_2014 to database...
Writing fm_2015 to database...
Writing fm_2016 to database...
Writing fm_2017 to database...
Writing fm_2018 to database...
Writing fm_2019 to database...
Writing fm_2020 to database...
Writing fm_2021 to database...
Writing fm_2022 to database...


In [ ]:
del fm_datasets

In [ ]:
#query that binds fm_2022 and fm_2021
query = "SELECT * FROM fm_2022 UNION ALL SELECT * FROM fm_2021;"

fm_21_22 = pd.read_sql_query(query, conn)
#save to csv


,Date,3ZIP,LSN,CLDS,CIR,ELTV,DDD,CS,FPD,FIRST_F,MD,CLTV,DTI,LTV,OIR,P_TYPE,OLT
0,202202,125,F22Q10000012,0,2.625,57,0,768,202203,0,203702,57,28,57,2.625,SF,180
1,202203,125,F22Q10000012,0,2.625,48,0,768,202203,0,203702,57,28,57,2.625,SF,180
2,202204,125,F22Q10000012,0,2.625,52,0,768,202203,0,203702,57,28,57,2.625,SF,180
3,202205,125,F22Q10000012,0,2.625,40,0,768,202203,0,203702,57,28,57,2.625,SF,180
4,202206,125,F22Q10000012,0,2.625,39,0,768,202203,0,203702,57,28,57,2.625,SF,180


# Merge ENSO data

In [ ]:
#Load enso_mei_long.csv
enso_mei_long = pd.read_csv("../Data/enso_mei_long.csv")
#Transform Month Dec to 12, Jan to 1, Feb to 2, etc.
enso_mei_long['Month'] = enso_mei_long['Month'].apply(extract_month)
enso_mei_long['Date'] = enso_mei_long.apply(lambda row: f"{row['Year'].astype(int)}{row['Month'].astype(int):02}", axis = 1)
enso_mei_long.head()

,Year,Month,MEI,Date
0,1979,12,0.47,197912
1,1979,1,0.27,197901
2,1979,2,-0.04,197902
3,1979,3,0.26,197903
4,1979,4,0.35,197904


In [ ]:
enso_mei_long = enso_mei_long[['Date', 'MEI']]

In [ ]:
enso_mei_long.to_sql('enso_mei_long', conn, if_exists = "replace", index = False)

540

## Add HURR data to database

In [13]:
import pandas as pd
hrcn_data = pd.read_csv('mainland_usa_gdf_HRCN.csv', dtype={'3ZIP':str})


In [14]:
hrcn_data.head()
hrcn_data_short = hrcn_data[['3ZIP', 'HRCN_RISKS', 'HRCN_RISKV', 'HRCN_EVNTS', 'HRCN_EALS']]

In [16]:
hrcn_data_short

,3ZIP,HRCN_RISKS,HRCN_RISKV,HRCN_EVNTS,HRCN_EALS
0,360,67.653656,1.014608e+06,13.0,66.364029
1,365,98.878421,1.895320e+08,50.0,99.005620
2,360,77.164648,3.079796e+06,21.0,72.805880
3,350,51.099148,2.226110e+05,11.0,49.070471
4,350,54.015253,3.010121e+05,6.0,52.399481
...,...,...,...,...,...
2219,531,17.900404,2.029927e+04,1.0,19.584955
2220,530,14.042171,1.429553e+04,0.0,17.769131
2221,531,24.001795,3.034120e+04,1.0,29.096412
2222,549,5.966801,5.715024e+03,1.0,5.879810


In [15]:
hrcn_data_short.to_sql('hrcn_data_short', conn, if_exists = "replace", index = False)

2224

# Close Database

In [10]:
conn.close()

NameError: name 'conn' is not defined

## Function to merge zip data to performance

In [ ]:
def merge_zip_data(perf_data, zip_data, perf_zip_col, zip_data_col):
    """
    Merge external data with the performance dataset based on the 3zip code.
    
    Parameters:
    - perf_data: The performance dataset.
    - zip_data: The external dataset with 3zip level information.
    - perf_zip_col: The zip column name in the performance dataset (might be 5-digit zip).
    - zip_data_col: The 3zip column name in the external dataset.
    
    Returns:
    - Merged dataset.
    """
    # Convert 5-digit zip code to 3zip format
    perf_data['3zip'] = perf_data[perf_zip_col].astype(str).str[:3]
    zip_data['3zip'] = zip_data[zip_data_col].astype(str).str[:3]
    
    # Merge datasets based on the 3zip code
    merged_data = pd.merge(perf_data, zip_data, left_on='3zip', right_on='3zip', how='left')
    
    return merged_data